# Tech Blues Capstone
## First Draft Notebook

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats
import acquire
import tyler_explore

## Project Overview

## Project Planning

## Data Acquisition

In [2]:
# use our 'get_survey_data' function to bring in the data
df = acquire.get_survey_data('survey.csv')

### Data Acquisition - Key Findings, Takeaways, and Next Steps:

## Data Preparation

### Data Preparation - Key Findings, Takeaways, and Next Steps:

## Data Exploration

### Univariate

#### Univariate - Key Findings, Takeaways, and Next Steps:

### Bivariate

#### Bivariate - Key Findings, Takeaways, and Next Steps:

### Multivariate

#### Multivariate - Key Findings, Takeaways, and Next Steps:

### Data Exploration - Key Findings, Takeaways, and Next Steps:

## Modeling

### Initital Setup

### Model 1: [insert model name here]

#### Model 1: [insert model name here] - Key Findings, Takeaways, and Next Steps:

### Model 2: [insert model name here]

#### Model 2: [insert model name here] - Key Findings, Takeaways, and Next Steps:

### Model 3: [insert model name here]

#### Model 3: [insert model name here] - Key Findings, Takeaways, and Next Steps:

### Model 4: [insert model name here]

#### Model 4: [insert model name here] - Key Findings, Takeaways, and Next Steps:

### Modeling - Key Findings, Takeaways, and Next Steps: